In [11]:
import ntsecuritycon, win32security, win32api
import webbrowser
import spotipy
import subprocess
import ctypes
import time
#checking process is running or not
def process_exists(process_name):
    call = 'TASKLIST', '/FI', 'imagename eq %s' % process_name
    # use buildin check_output right away
    output = subprocess.check_output(call).decode()
    # check in last line for process name
    last_line = output.strip().split('\r\n')[-1]
    # because Fail message could be translated
    return last_line.lower().startswith(process_name.lower())

# permission to change the timezone
def AdjustPrivilege(priv):
    flags = ntsecuritycon.TOKEN_ADJUST_PRIVILEGES | ntsecuritycon.TOKEN_QUERY
    htoken = win32security.OpenProcessToken(win32api.GetCurrentProcess(), flags)
    id = win32security.LookupPrivilegeValue(None, priv)
    newPrivileges = [(id, ntsecuritycon.SE_PRIVILEGE_ENABLED)]
    win32security.AdjustTokenPrivileges(htoken, 0, newPrivileges)


AdjustPrivilege(win32security.SE_TIME_ZONE_NAME)


#changing to pacific time zone
def pacific_time():
    print("pacific_time")
    win32api.SetTimeZoneInformation((480, u'Pacific Standard Time', (2000, 4, 1, 3, 0, 0, 0, 0), 0,
                                     u'Pacific Daylight Time', (2000, 10, 1, 2, 0, 0, 0, 0), -60))

#changing to original timezone
def main_time():
    print("main_time")
    win32api.SetTimeZoneInformation((-180,u'Turkey Standard Time',(0,0,0,0,0,0,0,0),0,
                                     u'Turkey DayLight Time',(0,0,0,0,0,0,0,0),-60))

#running all the users
def test_browsers():
    subprocess.run(r'"C:\Users\Efe\PycharmProjects\Voice_Assistant\Test_Pages.py"',shell=True)


#checking the paypal balance
def PayPal():
    webbrowser.open("https://www.paypal.com/us/signin")


def add_tracks_to_saved():
    print("add_tracks_to_saved")
    sp = spotipy.Spotify(auth=refresh())

    current = sp.current_user_playing_track()
    song_id = current["item"]["id"]
    current = sp.current_user_saved_tracks_add(tracks=[song_id])


def next_song():
    print("next_song")
    try:
        sp = spotipy.Spotify(auth=refresh())

        current = sp.next_track(device_id="")
    except:
        print('something went wrong')


def previous_song():
    print("previous_song")

    try:
        sp = spotipy.Spotify(auth=refresh())

        current = sp.previous_track(device_id="")
    except:
        print('something went wrong')

def start_playback():
    print("start_playback")

    if process_exists("Spotify.exe") == False:

        subprocess.run(r'"C:\Program Files\WindowsApps\SpotifyAB.SpotifyMusic_1.161.583.0_x86__zpdnekdrzrea0\Spotify.exe"')
        sp = spotipy.Spotify(auth=refresh())
        time.sleep(6)
        sp.start_playback(device_id="")
    else:
        sp = spotipy.Spotify(auth=refresh())
        sp.start_playback(device_id="")


def pause_playback():
    print("pause_playback")

    try:
        sp = spotipy.Spotify(auth=refresh())

        current = sp.pause_playback(device_id="")
    except:
        print('something went wrong')

def volume_level():
    sp = spotipy.Spotify(auth=refresh())
    sp.volume(20)

def lock_workstation():
    ctypes.windll.user32.LockWorkStation()



In [4]:
import requests
import json
spotify_user_id = ""
refresh_token = ""
base_64 = ""

Client_ID = ""
Client_Secret = ""

def refresh():
    query = "https://accounts.spotify.com/api/token"

    response = requests.post(query,
                             data={"grant_type": "refresh_token",
                                   "refresh_token": refresh_token},
                             headers={"Authorization": "Basic " + base_64})

    response_json = response.json()

    return response_json["access_token"]


# 


# 

In [5]:
import speech_recognition
from neuralintents import GenericAssistant
import pyttsx3 as tts
import spotipy
import re
import webbrowser
from playsound import playsound

C:\Users\Efe\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Efe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Efe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [ ]:
import speech_recognition

def take_command():
    recognizer = speech_recognition.Recognizer()

    with speech_recognition.Microphone() as mic:
        recognizer.adjust_for_ambient_noise(mic, duration=0.2)
        recognizer.dynamic_energy_threshold = True  
        audio = recognizer.listen(mic)

    try:
        print("recognizing")
        query = recognizer.recognize_google(audio, language="en-in").lower()

    except speech_recognition.UnknownValueError:
        print("not recognizied")
        return "None"

    return query




In [ ]:
def volume_level():
    print("volume")
    vol = ""
    try:
        vol = vol.join(filter(str.isdigit, query))
        sp = spotipy.Spotify(auth=refresh())
        sp.volume(int(vol))
    except:
        print("something went wrong")

def open():
    print("open")
    web = re.search('open (.+)', query)
    domain = web.group(1)
    url = "https://www." + domain + ".com"
    webbrowser.open(url)


mappings = {"test_browsers": test_browsers,
            "pacific": pacific_time,
            "main_time": main_time,
            "PayPal": PayPal,
            "add_tracks_to_saved": add_tracks_to_saved,
            "next_song": next_song,
            "previous_song": previous_song,
            "start_playback": start_playback,
            "pause_playback": pause_playback,
            "volume_level": volume_level,
            "open": open,
            "lock_workstation": lock_workstation}

assistant = GenericAssistant('intents.json', intent_methods=mappings)
assistant.load_model()

while True:
    print("1")
    query = take_command().lower()
    if "jarvis" in query:
        print(2)
        playsound("data/yes_sir_455.mp3")
        assistant.request(query)
        